In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
artworks = pd.read_csv('Artworks.csv')

In [4]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [5]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [6]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


In [7]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [8]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [9]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [10]:
for column in artworks:
    print(artworks[column].value_counts())

Multiple_Artists            5617
Louise Bourgeois            3230
Unknown photographer        2127
Lee Friedlander             1313
Pablo Picasso               1297
Marc Chagall                1151
Henri Matisse               1041
Jean Dubuffet               1022
Ludwig Mies van der Rohe     936
Pierre Bonnard               890
Émile Bernard                631
Georges Rouault              599
Aristide Maillol             576
André Derain                 567
Raoul Dufy                   534
Sol LeWitt                   527
Maurice Denis                500
Dorothea Lange               483
Joan Miró                    432
Jan Dibbets                  425
Pierre Alechinsky            417
Jasper Johns                 400
Jim Dine                     380
Unknown Artist               376
George Maciunas              366
Walker Evans                 358
Jules Pascin                 343
Garry Winogrand              342
Unknown Designer             340
Eugène Atget                 324
          

In [11]:
for column in X:
    print(X[column].value_counts())

True     64214
False    41817
Name: URL, dtype: int64
True     56465
False    49566
Name: ThumbnailURL, dtype: int64
0.000000      1046
21.000000      624
28.000000      557
5.080010       547
20.000000      547
15.000000      535
19.000000      535
17.780036      526
24.000000      523
30.500000      521
35.000000      499
19.100000      492
15.200000      490
27.940056      486
47.000000      473
20.300000      473
19.200000      465
17.800000      464
27.000000      429
33.000000      412
30.000000      411
21.600000      399
29.600000      393
23.000000      391
21.748793      386
29.500000      382
27.900000      377
27.400000      376
29.800000      371
18.000000      367
              ... 
109.900000       1
137.319025       1
249.200000       1
287.900000       1
242.253000       1
130.700000       1
156.845314       1
292.200000       1
107.632700       1
253.400000       1
431.800864       1
88.423900        1
29.972060        1
213.800000       1
217.487935       1
41.751300

0    105985
1        46
Name: (Romanian), dtype: int64
0    104197
1      1834
Name: (Russian), dtype: int64
0    105980
1        51
Name: (Scottish), dtype: int64
0    106030
1         1
Name: (Senegalese), dtype: int64
0    106030
1         1
Name: (Serbian), dtype: int64
0    106030
1         1
Name: (Singaporean), dtype: int64
0    106026
1         5
Name: (Slovak), dtype: int64
0    106005
1        26
Name: (Slovenian), dtype: int64
0    105692
1       339
Name: (South African), dtype: int64
0    103237
1      2794
Name: (Spanish), dtype: int64
0    106015
1        16
Name: (Sudanese), dtype: int64
0    105836
1       195
Name: (Swedish), dtype: int64
0    104399
1      1632
Name: (Swiss), dtype: int64
0    106028
1         3
Name: (Taiwanese), dtype: int64
0    106030
1         1
Name: (Tanzanian), dtype: int64
0    106008
1        23
Name: (Thai), dtype: int64
0    106018
1        13
Name: (Tunisian), dtype: int64
0    105987
1        44
Name: (Turkish), dtype: int64
0    106030

0    105661
1       370
Name: 2012, dtype: int64
0    105673
1       358
Name: 2013, dtype: int64
0    105839
1       192
Name: 2014, dtype: int64
0    105762
1       269
Name: 2015, dtype: int64
0    105807
1       224
Name: 2016, dtype: int64
0    105922
1       109
Name: 2017, dtype: int64
0    106029
1         2
Name: 2018, dtype: int64


# MLPC Configurations (adam):

### MLP 1000

In [12]:
artworks_reduced = artworks.sample(frac=.20, random_state=28)

In [13]:
artworks_reduced.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired
72008,José Clemente Orozco,(Mexican),(Male),1929,Prints & Illustrated Books,1940-04-05,True,True,28.300000,45.700000,1940
74662,Georges Vantongerloo,(Belgian),(Male),1921,Painting & Sculpture,1985-10-25,True,True,33.000000,37.500000,1985
53550,Jules Aarons,(American),(Male),1952,Photography,1965-11-09,True,True,39.200000,50.600000,1965
117713,Unknown photographer,(),(),1924,Photography,2010-10-07,False,False,14.605029,20.320041,2010
97219,George Maciunas,(American),(Male),1965,Prints & Illustrated Books,2008-10-08,False,False,0.000000,0.000000,2008


In [14]:
X_reduced = artworks_reduced.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists_reduced = pd.get_dummies(artworks_reduced.Artist)
nationalities_reduced = pd.get_dummies(artworks_reduced.Nationality)
dates_reduced = pd.get_dummies(artworks_reduced.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X_reduced = pd.get_dummies(X_reduced, sparse=True)
X_reduced = pd.concat([X_reduced, nationalities_reduced, dates_reduced], axis=1)

Y_reduced = artworks_reduced.Department

In [15]:
X_reduced

,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired,Gender_(),Gender_(Female),Gender_(Male),Gender_(male),Gender_\(multiple_persons\),...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
72008,True,True,28.300000,45.700000,1940,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
74662,True,True,33.000000,37.500000,1985,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
53550,True,True,39.200000,50.600000,1965,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
117713,False,False,14.605029,20.320041,2010,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97219,False,False,0.000000,0.000000,2008,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25405,True,True,4.200000,6.300000,1964,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
114388,False,False,23.500000,16.500000,2012,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85383,True,True,55.880100,76.200200,2005,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
58373,True,True,80.800000,60.000000,1994,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59584,True,True,21.500000,28.000000,1987,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [15]:
mlp.score(X_reduced, Y_reduced)

0.33580118834292183

In [16]:
Y_reduced.value_counts()/len(Y_reduced)

Prints & Illustrated Books    0.519664
Photography                   0.229558
Architecture & Design         0.114024
Drawings                      0.101999
Painting & Sculpture          0.034754
Name: Department, dtype: float64

In [17]:
from sklearn.model_selection import cross_val_score
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.58284233 0.467358   0.59288072 0.58999764 0.42793112]


0.5322019606761146


### MLP 316

In [18]:
# 316 features.
mlp = MLPClassifier(hidden_layer_sizes=(316,))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [19]:
mlp.score(X_reduced, Y_reduced)

0.5470621522210695

In [20]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.59368371 0.56870139 0.4339934  0.5881104  0.51285681]


0.5394691426932043


That's pretty inconsistent. Let's try increasing the layer size instead.

### MLP 3160,

In [21]:
# 316 times 10.
"""mlp = MLPClassifier(hidden_layer_sizes=(3160,))
mlp.fit(X_reduced, Y_reduced)"""

'mlp = MLPClassifier(hidden_layer_sizes=(3160,))\nmlp.fit(X_reduced, Y_reduced)'

In [22]:
"""mlp.score(X_reduced, Y_reduced)"""

'mlp.score(X_reduced, Y_reduced)'

In [23]:
"""cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())"""

"cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)\nprint(cvscore)\nprint('\n')\nprint(cvscore.mean())"

This is extremely computationally expensive, and for no gain. Let's go for depth instead of width.

### MLP 316, 5

In [24]:
mlp = MLPClassifier(hidden_layer_sizes=(316, 5))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [25]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [26]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51956624 0.51993395 0.51993395]


0.5196643430504748


### MLP 316, 316, 5

In [27]:
mlp = MLPClassifier(hidden_layer_sizes=(316, 316, 5))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316, 316, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [28]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [29]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51956624 0.51993395 0.50530786]


0.5167391248386324


### MLP 1000, 5

In [30]:
mlp = MLPClassifier(hidden_layer_sizes=(1000, 5))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [31]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [32]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51956624 0.51993395 0.51993395]


0.5196643430504748


### MLP 500, 500

In [69]:
mlp = MLPClassifier(hidden_layer_sizes=(500, 500))
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 500), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [70]:
mlp.score(X_reduced, Y_reduced)

0.282750165047628

In [71]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.61583785 0.55196795 0.55634135 0.55956594 0.61429582]


0.5796017812108328


# Solver:

### LBFGS 1000,

In [36]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,), solver='lbfgs')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [37]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [38]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51956624 0.51332862 0.51993395]


0.5183432767612557


### LBFGS 1000, 5

In [33]:
mlp = MLPClassifier(hidden_layer_sizes=(1000, 5), solver='lbfgs')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [34]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [35]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51980198 0.51993395 0.51993395]


0.5197114906223748


### LBFGS 316, 5

In [60]:
mlp = MLPClassifier(hidden_layer_sizes=(316, 5), solver='lbfgs')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [61]:
mlp.score(X_reduced, Y_reduced)

0.5196642459681222

In [62]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.51944379 0.51944379 0.51956624 0.51993395 0.51993395]


0.5196643430504748


# Activation Function:

### Activation Function: Logistic, 1000,

In [42]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [43]:
mlp.score(X_reduced, Y_reduced)

0.6282184287465812

In [44]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.6497761  0.64741928 0.65417256 0.65746638 0.66076905]


0.6539206747250367


### Activation Function: Logistic, 1000, 5

In [45]:
mlp = MLPClassifier(hidden_layer_sizes=(1000, 5), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [46]:
mlp.score(X_reduced, Y_reduced)

0.5397529001226068

In [47]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.54089088 0.63398539 0.64804338 0.55508375 0.63741448]


0.6030835746546607


### Activation Function: Logistic, 316, 316, 5

In [48]:
mlp = MLPClassifier(hidden_layer_sizes=(316, 316, 5), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316, 316, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [49]:
mlp.score(X_reduced, Y_reduced)

0.5299915118362728

In [50]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.54183361 0.63445675 0.54879774 0.59046945 0.5477707 ]


0.5726656496978945


### Activation Function: Logistic, 316, 5

In [51]:
mlp = MLPClassifier(hidden_layer_sizes=(316, 5), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(316, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [52]:
mlp.score(X_reduced, Y_reduced)

0.5528152409695369

In [53]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.64812633 0.53994815 0.63790665 0.55720689 0.65062515]


0.6067626318544066


In [54]:
mlp = MLPClassifier(hidden_layer_sizes=(1500,), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1500,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [55]:
mlp.score(X_reduced, Y_reduced)

0.6541544845798359

In [56]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.63563516 0.65166156 0.64545026 0.61948573 0.65888181]


0.6422229045683416


Hey! Those are some big improvements!

### Logistic, 100, 4 

In [57]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 4), activation='logistic')
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 4), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [58]:
mlp.score(X_reduced, Y_reduced)

0.6055833254739225

In [59]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.6271506  0.63704926 0.62918435 0.63906582 0.65038924]


0.6365678531504891


# Alpha:

### 1000, Alpha = 0.001

In [89]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,), alpha=0.001)
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [73]:
mlp.score(X_reduced, Y_reduced)

0.6169951900405546

In [80]:
mlp.score(X_reduced, Y_reduced)

0.5547958125058946

In [82]:
mlp.score(X_reduced, Y_reduced)

0.26435914363859286

In [84]:
mlp.score(X_reduced, Y_reduced)

0.19159671791002547

In [86]:
mlp.score(X_reduced, Y_reduced)

0.5971894746769782

In [88]:
mlp.score(X_reduced, Y_reduced)

0.616287843063284

In [90]:
mlp.score(X_reduced, Y_reduced)

0.33674431764594925

In [ ]:
mlp.score(X_reduced, Y_reduced)

In [74]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.56964412 0.47631393 0.55469118 0.6079264  0.59707478]


0.5611300822956162


### 1000, Alpha = 0.01

In [66]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,), alpha=0.01)
mlp.fit(X_reduced, Y_reduced)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [67]:
mlp.score(X_reduced, Y_reduced)

0.24639253041592002

In [68]:
cvscore = cross_val_score(mlp, X_reduced, Y_reduced, cv=5)
print(cvscore)
print('\n')
print(cvscore.mean())

[0.44921046 0.48032053 0.2932579  0.48785091 0.61854211]


0.4658363813323641


# Conclusions:

Despite the input size being 316, 1000 estimators does seem to produce higher scores. More layers tends to increase consistency of results.

Logisitic activation function with 1000 perceptrons as a single layer was the most effective model tested so far.